In [2]:
import pandas as pd

df = pd.read_csv('../responses/other/gpt4.1-all.csv')
df.shape

(6450, 7)

In [4]:
import pandas as pd 
import ast
from scipy.spatial import distance 

# AI wrote this function
def string_to_list_of_floats(s):
    if pd.isna(s): 
        return None
    try:
        evaluated_list = ast.literal_eval(s)
        if not isinstance(evaluated_list, list): 
             return None
        return [float(x) for x in evaluated_list]
    except (ValueError, SyntaxError, TypeError):
        return None

df['llm_distribution_val'] = df['llm_distribution'].apply(string_to_list_of_floats)
df['selections_val'] = df['selections'].apply(string_to_list_of_floats)


def compute_jsd_for_row(row):
    p = row['llm_distribution_val']
    q = row['selections_val']

    if p is None or q is None: 
        return float('nan')
    if not p or not q: 
        return float('nan')
    if len(p) != len(q): 
        return float('nan')
    if any(x < 0 for x in p) or any(x < 0 for x in q): 
        return float('nan')
    try:
        return distance.jensenshannon(p, q)
    except Exception:

        return float('nan')

df['jsd'] = df.apply(compute_jsd_for_row, axis=1)


/Users/andrew/GlobalOpinionBench/.venv/lib/python3.13/site-packages/scipy/spatial/distance.py:1382: RuntimeWarning: invalid value encountered in divide
  q = q / np.sum(q, axis=axis, keepdims=True)


In [44]:
df.iloc[6149]

Unnamed: 0                                                           6149
question                For each of the following aspects, indicate ho...
country                                                           Uruguay
selections              [0.163, 0.24100000000000002, 0.286000000000000...
options                 ['Very important', 'Rather important', 'Not ve...
llm_reasoning           Uruguay is known for its strong democratic tra...
llm_distribution                            [0.4, 0.35, 0.15, 0.08, 0.02]
llm_distribution_val                        [0.4, 0.35, 0.15, 0.08, 0.02]
selections_val          [0.163, 0.24100000000000002, 0.286000000000000...
jsd                                                                   NaN
Name: 6149, dtype: object

In [45]:
good_ones = df[df['jsd'] < .15]
good_ones.shape

(1824, 10)

In [46]:
good_ones.iloc[200]['llm_reasoning']

'Bosnia and Herzegovina has a complex political landscape marked by ethnic divisions, a history of conflict, and ongoing challenges with corruption and political influence. Many citizens are likely to perceive that wealthy individuals or groups have significant influence over elections, given the prominence of oligarchic structures and patronage networks in the country. \n\nCorruption and vote-buying have been reported issues in the region, and public trust in the electoral process is generally low. Therefore, a significant portion of respondents might say "Very often" or "Fairly often" to indicate that rich people buy elections. \n\nHowever, some respondents might be more cautious or skeptical, choosing "Not often" or "Not at all often," either due to lack of direct experience or political allegiance. A smaller portion might respond with "Don\'t know" due to uncertainty or lack of information. "No answer" and "Other missing" categories are typically small.\n\nBased on this reasoning, 

In [47]:
def generate_prompt(row):
    question = row['question']
    country = row['country']
    options = row['options']
    prompt = (
        f"Given the following question and answer options, estimate the distribution of responses you would expect from people in {country}. "
        f"Respond with your reasoning in <think></think> tags, then output your estimated distribution as a list of numbers that sum to one in the format {{X, Y, Z, ...}}.\n\n"
        f"Question: {question}\n"
        f"Options: {options}\n"
        f"Country: {country}\n\n"
        f"<think>Think step-by-step about cultural, historical, and social factors that might influence how people in {country} would answer this question.</think>\n"
        f"Distribution: {{X, Y, Z, ...}}"
    )
    return prompt

In [48]:
import pandas as pd
import ast

def generate_answer(row):
    llm_reasoning = row['llm_reasoning']
    llm_distribution_str = row['llm_distribution']
    
    if not isinstance(llm_distribution_str, str):
        llm_distribution_str = str(llm_distribution_str)

    if llm_distribution_str.startswith('[') and llm_distribution_str.endswith(']'):
        formatted_distribution = '{' + llm_distribution_str[1:-1] + '}'
    else:
        formatted_distribution = f"{{{llm_distribution_str}}}"

    return f"<think>{llm_reasoning}</think> {formatted_distribution}"

In [49]:
good_ones['prompt'] = good_ones.apply(generate_prompt, axis=1)
good_ones['answer'] = good_ones.apply(generate_answer, axis=1)
good_ones.head()

/var/folders/nk/5zcc814x561cbxxgrncjyscw0000gn/T/ipykernel_90734/968541182.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_ones['prompt'] = good_ones.apply(generate_prompt, axis=1)
/var/folders/nk/5zcc814x561cbxxgrncjyscw0000gn/T/ipykernel_90734/968541182.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  good_ones['answer'] = good_ones.apply(generate_answer, axis=1)


,Unnamed: 0,question,country,selections,options,llm_reasoning,llm_distribution,llm_distribution_val,selections_val,jsd,prompt,answer
4,4,I'm going to read out a list of various change...,Albania,"[0.309, 0.189, 0.466, 0.028999999999999998, 0....","['Good thing', ""Don't mind"", 'Bad thing', ""Don...","Albania is a country with a strong work ethic,...","[0.25, 0.15, 0.5, 0.05, 0.03, 0.02]","[0.25, 0.15, 0.5, 0.05, 0.03, 0.02]","[0.309, 0.189, 0.466, 0.028999999999999998, 0....",0.118448,Given the following question and answer option...,<think>Albania is a country with a strong work...
9,9,On this list are various groups of people. Cou...,Albania,"[0.153, 0.8240000000000001, 0.012, 0.01, 0.001]","['Mentioned', 'Not mentioned', ""Don't know"", '...",Albania is a relatively homogeneous country et...,"[0.3, 0.65, 0.03, 0.02, 0.0]","[0.3, 0.65, 0.03, 0.02, 0.0]","[0.153, 0.8240000000000001, 0.012, 0.01, 0.001]",0.143107,Given the following question and answer option...,<think>Albania is a relatively homogeneous cou...
14,14,I am going to name a number of organizations. ...,Albania,"[0.24800000000000003, 0.41700000000000004, 0.2...","['A great deal', 'Quite a lot', 'Not very much...",Albania has a history of political instability...,"[0.25, 0.35, 0.25, 0.1, 0.03, 0.02, 0.0]","[0.25, 0.35, 0.25, 0.1, 0.03, 0.02, 0.0]","[0.24800000000000003, 0.41700000000000004, 0.2...",0.076465,Given the following question and answer option...,<think>Albania has a history of political inst...
16,16,In which of the following things do you believ...,Albania,"[0.742, 0.22, 0.028999999999999998, 0.00699999...","['Yes', 'No', ""Don't know"", 'No answer', 'Othe...",Albania has a unique religious landscape shape...,"[0.68, 0.18, 0.12, 0.01, 0.01]","[0.68, 0.18, 0.12, 0.01, 0.01]","[0.742, 0.22, 0.028999999999999998, 0.00699999...",0.137403,Given the following question and answer option...,<think>Albania has a unique religious landscap...
19,19,Now I'd like you to look at this card. I'm goi...,Albania,"[0.294, 0.299, 0.37200000000000005, 0.02600000...","['Have done', 'Might do', 'Would never do', ""D...",Albania is a country with a relatively young d...,"[0.25, 0.4, 0.3, 0.03, 0.02, 0.0]","[0.25, 0.4, 0.3, 0.03, 0.02, 0.0]","[0.294, 0.299, 0.37200000000000005, 0.02600000...",0.096134,Given the following question and answer option...,<think>Albania is a country with a relatively ...


In [50]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

if 'prompt' not in good_ones.columns or 'answer' not in good_ones.columns:
    raise ValueError("DataFrame 'good_ones' must contain 'prompt' and 'answer' columns.")

train_df, eval_df = train_test_split(good_ones, test_size=0.1, random_state=42)

def df_to_jsonl(df, filename, prompt_col='prompt', completion_col='answer'):
    with open(filename, 'w') as f:
        for index, row in df.iterrows():
            json_record = {
                "prompt": row[prompt_col],
                "completion": row[completion_col]
            }
            f.write(json.dumps(json_record) + '\n')
    print(f"Successfully wrote {len(df)} records to {filename}")

train_jsonl_file = 'train.jsonl'
eval_jsonl_file = 'eval.jsonl'

df_to_jsonl(train_df, train_jsonl_file, prompt_col='prompt', completion_col='answer')
df_to_jsonl(eval_df, eval_jsonl_file, prompt_col='prompt', completion_col='answer')

Successfully wrote 1641 records to train.jsonl
Successfully wrote 183 records to eval.jsonl

Training data shape: (1641, 12)
Evaluation data shape: (183, 12)
Training data saved to train.jsonl
Evaluation data saved to eval.jsonl


In [5]:
high_quality_rows = df[df['jsd'] < 0.15]

not_in_training = df[df['jsd'] >= 0.15]

not_in_training.to_csv('not_in_training.csv', index=False)

print(f"Total rows in original dataframe: {len(df)}")
print(f"High-quality rows (JSD < 0.15): {len(high_quality_rows)}")
print(f"Rows not in training (JSD >= 0.15): {len(not_in_training)}")
print(f"Saved {len(not_in_training)} rows to not_in_training.csv")

Total rows in original dataframe: 6450
High-quality rows (JSD < 0.15): 1824
Rows not in training (JSD >= 0.15): 4305
Saved 4305 rows to not_in_training.csv
